In [2]:
from deepagents import create_deep_agent
from langchain_ollama import ChatOllama
# from langchain_openai import ChatOpenAI
from  langgraph.checkpoint.memory import MemorySaver
from dotenv import load_dotenv
load_dotenv()
import os

In [3]:
from typing import Literal
from tavily import TavilyClient
from langchain_core.tools import tool


tavily_client = TavilyClient()
@tool
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """
    Search the internet for information.
    
    Args:
        query: The search query string
        max_results: The maximum number of search results to return

        
    Returns:
        Search results as a dict
    """
    return tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )

In [4]:
llm = ChatOllama(
        api_key=os.environ.get("OLLAMA_API_KEY"),
        model="gpt-oss:120b",
        base_url="https://api.ollama.com",
        temperature=0.0,
    )
# llm = ChatOllama(
#     model="gpt-oss:120b-cloud", # Use the model you pulled
#     base_url="http://localhost:11434", # Point to the local Ollama server
#     temperature=0
# )

In [5]:
# prompt = open("../promt.md","r",encoding="utf8").read()
agent_graph = create_deep_agent(
    model=llm,
    tools=[internet_search],
    # system_prompt=prompt,
    checkpointer=MemorySaver(), # This enables Short-Term Chat Memory
    # response_format={"content":"answer of asked response","tools_used":["tool1","tool2"]}
)

In [6]:
config = {"configurable": {"thread_id": "session_1"}}

In [7]:
events = agent_graph.stream(
    {"messages": [("user", "write a plan to search who is prime minister of india using given tools")]},
    config,
    stream_mode="values",
    
)

In [1]:
final_answer = ""
for event in events:
    if "messages" in event:
        last_msg = event["messages"][-1]
        # Filter out intermediate tool calls, show only AI response
        if last_msg.type == "ai" and not last_msg.tool_calls:
            final_answer = last_msg.content

NameError: name 'events' is not defined

In [17]:
print(final_answer)

**Plan for Finding the Current Prime Minister of India Using the Available Tools**

| Step | Action | Tool to Use | Reason |
|------|--------|-------------|--------|
| 1 | Formulate a concise search query that directly asks for the current Prime Minister of India. | `internet_search` | The web is the most up‑to‑date source for political office holders. |
| 2 | Run the internet search with the query “who is the prime minister of India?” (or “current prime minister of India”). Request at least a few results (e.g., `max_results: 3`) and ask for raw content so the answer text is returned. | `internet_search` ( `query: "who is the prime minister of India?"`, `max_results: 3`, `include_raw_content: true` ) | This returns snippets from reliable sites (government pages, major news outlets, Wikipedia) that contain the name. |
| 3 | Inspect the returned results and locate the line or snippet that explicitly states the name of the Prime Minister. | – (manual inspection of the returned JSON) | The

In [10]:
for i in agent_graph.checkpointer.list(config):
    print(i[4],"\n")

[] 

[('8b05af2f-1c49-9939-881c-8b1f772aaef0', '__no_writes__', None)] 

[('89a09f9d-5eae-feec-007e-12fd708c2038', 'messages', [AIMessage(content='The Prime Minister of India is **Narendra\u202fModi**. He has been in office since May\u202f2014.', additional_kwargs={}, response_metadata={'model': 'gpt-oss:120b', 'created_at': '2026-02-14T07:30:48.192643049Z', 'done': True, 'done_reason': 'stop', 'total_duration': 799386011, 'load_duration': None, 'prompt_eval_count': 4528, 'prompt_eval_duration': None, 'eval_count': 75, 'eval_duration': None, 'logprobs': None, 'model_name': 'gpt-oss:120b', 'model_provider': 'ollama'}, id='lc_run--019c5b0f-245b-7dc1-85f4-8b5d4ed14433-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 4528, 'output_tokens': 75, 'total_tokens': 4603})]), ('89a09f9d-5eae-feec-007e-12fd708c2038', 'branch:to:TodoListMiddleware.after_model', None)] 

[('2dc18c99-074e-6850-70a5-ac507831fd5c', 'branch:to:model', None)] 

[('66bfe251-5ca3-bf64-0c84-34418291